Lazy loading propositions proof from set.mm so we can serve specific theorems without having to load the entire library.

http://us.metamath.org/mpeuni/epee.html

http://us.metamath.org/mpeuni/opoeALTV.html

http://us.metamath.org/mpeuni/elun.html

http://us.metamath.org/mpegif/oddm1evenALTV.html

http://us.metamath.org/mpegif/oddp1evenALTV.html

http://us.metamath.org/mpegif/2evenALTV.html

In [1]:
from adapt_utils import replace_symbols

from tree_parser import *

import networkx as nx

import matplotlib
import matplotlib.pyplot as plt

import copy

import time

from adapt_utils import replace_symbols
from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
from my_utils import get_proof_steps_graph, print_proof_steps_graph

In [3]:
%%time

text = file_contents()
database = meta_math_database(text,n=20000, remember_proof_steps=True)

included 5555695 tokens from set.mm
proposition: 16100

Exception: ('Parsing of', ['|-', '.o.', '=', '(', '<.', 'X', ',', 'Y', '>.', '(', 'comp', '`', 'C', ')', 'X', ')'], 'failed')

In [2]:
f = open("set.mm", 'r')
f_contents = f.read().split()

In [16]:
propositions = []
propositions_idx = []
idx2prop = {}
prop2idx = {}

for i, token in enumerate(f_contents):
    if token == "$p":
        prop = f_contents[i-1]
        propositions_idx.append(i-1)
        propositions.append(prop)
        idx2prop[i-1] = prop
        prop2idx[prop] = i-1 

In [18]:
prop2idx["opoeALTV"]

5243177

In [19]:
f_contents[5243177]

'opoeALTV'

In [31]:
def get_proof_from_prop(prop):
    prop_idx = prop2idx[prop]
    
    token_id = prop_idx
    token = f_contents[token_id]
    
    start_proof = False
    proof_tokens = []
    
    while token != "$.":        
        if token == "$=":
            start_proof = True
        
        elif start_proof:
            proof_tokens.append(token)
            
        token_id += 1
        token = f_contents[token_id]
            
    return proof_tokens    

def split_compressed_proof(list):
    assert list[0] == '('
    index = 1
    while list[index] != ')': index+=1
    labels = list[1:index]  # not including index
    proof = ''
    for part in list[index+1:]: proof+=part
    return labels, proof

In [36]:
proof_tokens = get_proof_from_prop('bitri')
#proof_tokens

In [37]:
labels, proof = split_compressed_proof(proof_tokens)
labels, proof

(['sylbb', 'sylbbr', 'impbii'], 'ACABCDEFABCDEGH')

In [ ]:
def uncompress(self, prop, context):

    compressed_proof=prop.proof
    if compressed_proof[0]!='(': return # don't do anything if not compressed

    label_part, proof = split_compressed_proof(compressed_proof)

    # construct a list of all of the valid labels and their arities
    # we'll need this when expanding Z statements in the compressed proof.
    arities = {}
    hypothesis_labels = [hyp.label for hyp in prop.hyps]
    for label in hypothesis_labels:
        arities[label]=0

    token_list = hypothesis_labels+label_part
    for label in label_part:
        if label in self.propositions:
            arities[label] = self.propositions[label].arity()
        elif label in context.f:
            arities[label] = 0 # hypotheses have 0 arity
        else:
            print(prop.print_details())
            print(token_list)
            print(proof)
            print(label)
            raise Exception('not defined')
    zpositions = []

    proof = proof.upper()
    out = []
    index = -1
    current = 0
    for char in proof:
        index+=1 # not needed.  kept for reference
        num = ord(char)
        if num==90: # Z
            zpositions.append(len(out)-1) #save the last one as a z-position
        elif (85<=num<=89): # Ord('U')=85, Ord('Y')=89
            current *= 5
            current += num-84
        else:  # Ord('A')=65
            current = current *20 + num-64

            # append to the out list the corresponding stuff
            if current<=len(token_list):
                out.append(token_list[current-1])
            elif current - len(token_list)<=len(zpositions):
                # pull a bunch of stuff from the previous position
                source_position = zpositions[current - len(token_list)-1]
                to_append = [] # the new stuff to append, written backwards
                remaining_arity = 1;
                while remaining_arity>0:
                    if source_position<0:
                        print(token_list,zpositions,compressed_proof)
                        print(to_append,source_position)
                        print(remaining_arity)
                        prop.print_details()
                        raise Exception('reached start of proof before finishing Z substitution')
                    new_token = out[source_position]
                    remaining_arity += -1 + arities[new_token]
                    to_append.append(new_token)
                    source_position-=1
                to_append.reverse()
                out += to_append
            else:
                print(len(token_list),len(zpositions), 'current=',current)
                print(token_list,proof[index-5:index+1])
                raise Exception('Attemped to refer to non-existant token position')

            # now reset the counter
            current = 0
    prop.proof = out # replace the compressed proof with the uncompressed one
    prop._uncompressed = True